In [287]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklift.metrics import uplift_at_k, qini_auc_score
from sklift.models import SoloModel
from catboost import CatBoostClassifier

In [288]:
clients = pd.read_csv('uplift-focus/x5-uplift-valid/data/clients2.csv').set_index('client_id')
products = pd.read_csv('uplift-focus/x5-uplift-valid/data/products.csv')
train = pd.read_csv('uplift-focus/x5-uplift-valid/data/train.csv').set_index('client_id')
test = pd.read_csv('uplift-focus/x5-uplift-valid/data/test.csv')


In [289]:
train_purch = pd.read_csv('uplift-focus/x5-uplift-valid/train_purch/train_purch.csv')
test_purch = pd.read_csv('uplift-focus/x5-uplift-valid/test_purch/test_purch.csv')

In [290]:
test_purch

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,a375a035b9,2.0,50.0,NaN
1,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,6f9e02ee36,1.0,90.0,NaN
2,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,42e1eb8d23,1.0,20.0,NaN
3,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,5c6de3010f,2.0,85.0,NaN
4,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,343e841aaa,1.0,58.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6883733,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,24d5a54f42,1.0,164.0,NaN
6883734,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,7427d0830d,1.0,134.0,NaN
6883735,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,6c547c2ca8,1.0,60.0,NaN
6883736,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,d04e7a8338,1.0,48.0,NaN


In [291]:
clients.head()

,client_id.1,first_issue_date,first_redeem_date,age,gender
client_id,,,,,
000012768d,NaN,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
000036f903,NaN,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
00010925a5,NaN,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U
0001f552b0,NaN,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
00020e7b18,NaN,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U


In [292]:
clients.drop('client_id.1', axis = 1, inplace = True)

In [293]:
clients.describe()

,age
count,200039.000000
mean,46.417329
std,49.532475
min,-7491.000000
25%,34.000000
50%,45.000000
75%,59.000000
max,1852.000000


In [294]:
q_upper = np.quantile(clients['age'],0.99)
q_lower = np.quantile(clients['age'],0.01)

clients_age_cleaned = clients[(clients['age'] < q_upper) & (clients['age'] > q_lower)]['age']

In [295]:
clients['first_issue_date'] = pd.to_datetime(clients['first_issue_date'])
clients['first_issue_date_d'] = clients['first_issue_date'].dt.date
clients_issues_dates = pd.DataFrame(clients.groupby('first_issue_date_d').size()).\
reset_index().rename(columns = {0: 'Number of clients'})
clients

,first_issue_date,first_redeem_date,age,gender,first_issue_date_d
client_id,,,,,
000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U,2017-08-05
000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F,2017-04-10
00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U,2018-07-24
0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F,2017-06-30
00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U,2017-11-27
...,...,...,...,...,...
fffe0abb97,2017-11-27 08:56:54,2018-02-11 09:26:08,35,F,2017-11-27
fffe0ed719,2017-09-15 08:53:24,2017-12-12 14:50:12,69,U,2017-09-15
fffea1204c,2018-01-31 16:59:37,2018-03-12 17:02:27,73,F,2018-01-31


In [296]:
train_purch.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


In [297]:
clients['first_issue_date'] = pd.to_datetime(clients['first_issue_date'])
clients['first_issue_date_d'] = clients['first_issue_date'].dt.date
clients_issues_dates = pd.DataFrame(clients.groupby('first_issue_date_d').size()).\
reset_index().rename(columns = {0: 'Number of clients'})

In [298]:
train

,treatment_flg,purchased
client_id,,
ad6561e2d8,1,1
7c1ccbf93f,1,1
b58fadcab6,1,1
e99e6fabb9,0,0
27fb6f8520,1,1
...,...,...
999d284453,1,1
f634deea4e,0,1
16cb4f99b0,0,1


In [299]:
train_df = clients.merge(train,how = 'right', on = 'client_id')
test_df = clients.merge(test, how  = 'right', on = 'client_id')
test_df

,client_id,first_issue_date,first_redeem_date,age,gender,first_issue_date_d
0,a9a604ed6e,2018-09-13 17:36:41,NaN,36,F,2018-09-13
1,ebd7360016,2017-07-03 17:12:12,2017-09-01 16:15:12,63,F,2017-07-03
2,908cd9b8e8,2018-03-30 17:01:20,2018-07-13 17:05:52,49,F,2018-03-30
3,dceb8ce861,2017-10-06 17:06:42,2018-08-11 18:43:58,46,U,2017-10-06
4,f4f0ac6b06,2018-06-27 21:39:41,2019-02-15 20:14:37,119,U,2018-06-27
...,...,...,...,...,...,...
60007,4762fb04c5,2018-05-31 16:03:28,2018-12-26 14:03:30,26,M,2018-05-31
60008,533d1516e7,2017-09-29 08:27:16,2018-01-07 17:36:31,37,M,2017-09-29
60009,c93b21a707,2018-10-03 21:02:26,2019-03-30 17:40:42,42,U,2018-10-03
60010,3fe0668de5,2018-12-25 18:27:14,2019-02-02 10:30:53,33,U,2018-12-25


In [300]:
test_df

,client_id,first_issue_date,first_redeem_date,age,gender,first_issue_date_d
0,a9a604ed6e,2018-09-13 17:36:41,NaN,36,F,2018-09-13
1,ebd7360016,2017-07-03 17:12:12,2017-09-01 16:15:12,63,F,2017-07-03
2,908cd9b8e8,2018-03-30 17:01:20,2018-07-13 17:05:52,49,F,2018-03-30
3,dceb8ce861,2017-10-06 17:06:42,2018-08-11 18:43:58,46,U,2017-10-06
4,f4f0ac6b06,2018-06-27 21:39:41,2019-02-15 20:14:37,119,U,2018-06-27
...,...,...,...,...,...,...
60007,4762fb04c5,2018-05-31 16:03:28,2018-12-26 14:03:30,26,M,2018-05-31
60008,533d1516e7,2017-09-29 08:27:16,2018-01-07 17:36:31,37,M,2017-09-29
60009,c93b21a707,2018-10-03 21:02:26,2019-03-30 17:40:42,42,U,2018-10-03
60010,3fe0668de5,2018-12-25 18:27:14,2019-02-02 10:30:53,33,U,2018-12-25


In [301]:
train_df['first_issue_time'] = \
    (pd.to_datetime(train_df['first_issue_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


test_df['first_issue_time'] = \
    (pd.to_datetime(test_df['first_issue_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


train_df['first_redeem_time'] = \
    (pd.to_datetime(train_df['first_redeem_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


test_df['first_redeem_time'] = \
    (pd.to_datetime(test_df['first_redeem_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


train_df['issue_redeem_delay'] = train_df['first_redeem_time'] \
    - train_df['first_issue_time']


test_df['issue_redeem_delay'] = test_df['first_redeem_time'] \
    - test_df['first_issue_time']


In [302]:
purchases_train = train_purch.merge(train,how = 'right', on = 'client_id')
purchases_features = purchases_train.groupby('client_id')[['transaction_id','product_id']].nunique().reset_index()

purchases_test = test_purch.merge(test_df,how = 'right', on = 'client_id')
purchases_test_features = purchases_test.groupby('client_id')[['transaction_id','product_id']].nunique().reset_index()
purchases_test_features

,client_id,transaction_id,product_id
0,00010925a5,18,58
1,00035a21d9,3,25
2,00038f9200,48,100
3,0004315e57,14,93
4,0006fca4bf,3,28
...,...,...,...
60007,fff9aa288c,10,51
60008,fff9f772f6,4,77
60009,fffe0ed719,30,89
60010,fffea1204c,17,45


In [303]:
purchases_train

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,treatment_flg,purchased
0,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,1bc3da0e85,1.0,40.0,NaN,1,1
1,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,4752aa4b20,1.0,55.0,NaN,1,1
2,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,3d7742b802,1.0,100.0,NaN,1,1
3,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,c89a550a15,1.0,50.0,NaN,1,1
4,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,5c44498721,1.0,25.0,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15998947,1430184499,c55462da76,2019-03-15 16:32:10,2.6,0.0,0.0,0.0,526.39,ebf9499b63,5d125fa59a,1.0,120.0,NaN,0,1
15998948,1430184499,c55462da76,2019-03-15 16:32:10,2.6,0.0,0.0,0.0,526.39,ebf9499b63,67e5809518,1.0,32.0,NaN,0,1
15998949,1430184499,c55462da76,2019-03-15 16:32:10,2.6,0.0,0.0,0.0,526.39,ebf9499b63,a449f77e31,1.0,33.0,NaN,0,1
15998950,1430184499,c55462da76,2019-03-15 16:32:10,2.6,0.0,0.0,0.0,526.39,ebf9499b63,29de99801b,1.0,33.0,NaN,0,1


In [304]:
train_df_purchases = train_df.merge(purchases_features, how = 'left', on = 'client_id')
train_df_purchases


test_df_purchases = test_df.merge(purchases_test_features, how = 'left', on = 'client_id')
test_df_purchases

,client_id,first_issue_date,first_redeem_date,age,gender,first_issue_date_d,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id
0,a9a604ed6e,2018-09-13 17:36:41,NaN,36,F,2018-09-13,1536860201,NaN,NaN,5,41
1,ebd7360016,2017-07-03 17:12:12,2017-09-01 16:15:12,63,F,2017-07-03,1499101932,1.504283e+09,5180580.0,31,68
2,908cd9b8e8,2018-03-30 17:01:20,2018-07-13 17:05:52,49,F,2018-03-30,1522429280,1.531502e+09,9072272.0,17,75
3,dceb8ce861,2017-10-06 17:06:42,2018-08-11 18:43:58,46,U,2017-10-06,1507309602,1.534013e+09,26703436.0,6,93
4,f4f0ac6b06,2018-06-27 21:39:41,2019-02-15 20:14:37,119,U,2018-06-27,1530135581,1.550262e+09,20126096.0,12,91
...,...,...,...,...,...,...,...,...,...,...,...
60007,4762fb04c5,2018-05-31 16:03:28,2018-12-26 14:03:30,26,M,2018-05-31,1527782608,1.545833e+09,18050402.0,30,127
60008,533d1516e7,2017-09-29 08:27:16,2018-01-07 17:36:31,37,M,2017-09-29,1506673636,1.515347e+09,8672955.0,10,14
60009,c93b21a707,2018-10-03 21:02:26,2019-03-30 17:40:42,42,U,2018-10-03,1538600546,1.553968e+09,15367096.0,34,166
60010,3fe0668de5,2018-12-25 18:27:14,2019-02-02 10:30:53,33,U,2018-12-25,1545762434,1.549103e+09,3341019.0,14,40


In [305]:
test_df_purchases.drop('client_id', axis = 1, inplace = True)

In [306]:
test_df_purchases

,first_issue_date,first_redeem_date,age,gender,first_issue_date_d,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id
0,2018-09-13 17:36:41,NaN,36,F,2018-09-13,1536860201,NaN,NaN,5,41
1,2017-07-03 17:12:12,2017-09-01 16:15:12,63,F,2017-07-03,1499101932,1.504283e+09,5180580.0,31,68
2,2018-03-30 17:01:20,2018-07-13 17:05:52,49,F,2018-03-30,1522429280,1.531502e+09,9072272.0,17,75
3,2017-10-06 17:06:42,2018-08-11 18:43:58,46,U,2017-10-06,1507309602,1.534013e+09,26703436.0,6,93
4,2018-06-27 21:39:41,2019-02-15 20:14:37,119,U,2018-06-27,1530135581,1.550262e+09,20126096.0,12,91
...,...,...,...,...,...,...,...,...,...,...
60007,2018-05-31 16:03:28,2018-12-26 14:03:30,26,M,2018-05-31,1527782608,1.545833e+09,18050402.0,30,127
60008,2017-09-29 08:27:16,2018-01-07 17:36:31,37,M,2017-09-29,1506673636,1.515347e+09,8672955.0,10,14
60009,2018-10-03 21:02:26,2019-03-30 17:40:42,42,U,2018-10-03,1538600546,1.553968e+09,15367096.0,34,166
60010,2018-12-25 18:27:14,2019-02-02 10:30:53,33,U,2018-12-25,1545762434,1.549103e+09,3341019.0,14,40


In [307]:
selected_features = ['age','gender','first_issue_time','first_redeem_time','issue_redeem_delay',
                     'transaction_id','product_id', 'treatment_flg']
y = train_df_purchases['purchased']


In [308]:
train_df_purchases[selected_features]

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id,treatment_flg
0,50,F,1512322431,1.527102e+09,14779510.0,40,135,1
1,24,F,1510331629,1.519326e+09,8994516.0,2,14,1
2,36,U,1509657465,1.537045e+09,27387871.0,33,117,1
3,79,F,1526466080,1.527713e+09,1247284.0,13,38,0
4,34,F,1500320188,1.513332e+09,13012137.0,30,67,1
...,...,...,...,...,...,...,...,...
140022,72,U,1540991437,1.554559e+09,13567144.0,4,53,1
140023,35,U,1517752292,1.520877e+09,3124210.0,53,130,0
140024,50,F,1519380153,1.559324e+09,39944103.0,7,33,0
140025,55,U,1498935933,1.508353e+09,9416669.0,17,40,1


In [309]:
X_train, X_val, y_train, y_val = train_test_split(train_df_purchases[selected_features],y,
                                                  test_size=0.3, random_state=123)
X_train

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id,treatment_flg
35393,18,M,1503484834,1.558094e+09,54608948.0,12,40,1
63384,32,M,1496267338,1.498257e+09,1989456.0,41,188,1
48777,78,U,1503919951,1.505240e+09,1320149.0,4,9,1
91057,64,F,1513442311,1.518542e+09,5099581.0,109,183,1
33732,59,F,1491902582,1.529940e+09,38037917.0,18,57,1
...,...,...,...,...,...,...,...,...
129130,41,U,1501271422,1.502131e+09,859616.0,21,101,1
119906,56,U,1501262081,1.520025e+09,18763228.0,6,136,1
17730,33,U,1499430832,1.530559e+09,31128098.0,24,94,0
28030,61,U,1496052246,1.513005e+09,16952991.0,23,75,1


In [310]:
treatment_train = X_train['treatment_flg']
treatment_val = X_val['treatment_flg']

X_train = X_train.drop(columns = ['treatment_flg'])
X_val = X_val.drop(columns = ['treatment_flg'])

In [311]:
estimator = CatBoostClassifier(verbose=100, 
                               cat_features=['gender'],
                               random_state=42,
                               thread_count=1)

sm = SoloModel(estimator)
sm = sm.fit(X_train, y_train, treatment_train)

Learning rate set to 0.072983
0:	learn: 0.6731588	total: 85.3ms	remaining: 1m 25s
100:	learn: 0.5466868	total: 8.31s	remaining: 1m 13s
200:	learn: 0.5433871	total: 17.1s	remaining: 1m 8s
300:	learn: 0.5399585	total: 25.3s	remaining: 58.8s
400:	learn: 0.5368827	total: 33.4s	remaining: 49.8s
500:	learn: 0.5342719	total: 41.5s	remaining: 41.4s
600:	learn: 0.5316905	total: 49.6s	remaining: 32.9s
700:	learn: 0.5292868	total: 57.7s	remaining: 24.6s
800:	learn: 0.5268832	total: 1m 6s	remaining: 16.4s
900:	learn: 0.5245934	total: 1m 14s	remaining: 8.16s
999:	learn: 0.5224380	total: 1m 23s	remaining: 0us


In [312]:
X_val

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id
78102,52,U,1501280654,1.526239e+09,24958695.0,26,84
137586,61,F,1518799975,1.521827e+09,3027006.0,20,118
118419,24,F,1540548503,1.544459e+09,3910280.0,21,68
122612,29,F,1535654421,1.537968e+09,2313608.0,21,51
20207,69,U,1498307006,1.507643e+09,9336305.0,37,113
...,...,...,...,...,...,...,...
12449,66,M,1502992616,1.553365e+09,50372236.0,16,67
20242,62,U,1536679872,1.549457e+09,12777026.0,16,112
79587,15,F,1534774667,1.534775e+09,-13.0,13,64
99420,38,U,1518358648,1.536694e+09,18335329.0,11,56


In [328]:
uplift_sm = sm.predict(X_val)
sm_score =  qini_auc_score(y_true=y_val, uplift=uplift_sm, treatment=treatment_val)

In [333]:
uplift_sm

array([0.01970905, 0.04111449, 0.02537724, ..., 0.094332  , 0.02363439,
       0.05310498])

In [336]:
uplift_sm[uplift_sm < 0]

array([-0.0099069 , -0.00575619, -0.03732471, ..., -0.034677  ,
       -0.00954834, -0.00513308])

In [323]:
train_df_purchases[selected_features]

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id,treatment_flg
0,50,F,1512322431,1.527102e+09,14779510.0,40,135,1
1,24,F,1510331629,1.519326e+09,8994516.0,2,14,1
2,36,U,1509657465,1.537045e+09,27387871.0,33,117,1
3,79,F,1526466080,1.527713e+09,1247284.0,13,38,0
4,34,F,1500320188,1.513332e+09,13012137.0,30,67,1
...,...,...,...,...,...,...,...,...
140022,72,U,1540991437,1.554559e+09,13567144.0,4,53,1
140023,35,U,1517752292,1.520877e+09,3124210.0,53,130,0
140024,50,F,1519380153,1.559324e+09,39944103.0,7,33,0
140025,55,U,1498935933,1.508353e+09,9416669.0,17,40,1


In [324]:
selected_features_for_predict = ['age','gender','first_issue_time','first_redeem_time','issue_redeem_delay',
                     'transaction_id','product_id']
uplift_sm = sm.predict(test_df_purchases[selected_features_for_predict])
uplift_sm

array([ 0.00152367,  0.02892214,  0.03589107, ..., -0.01209885,
        0.05130666,  0.0313057 ])

In [325]:
test['pred'] = uplift_sm
test

,client_id,pred
0,a9a604ed6e,0.001524
1,ebd7360016,0.028922
2,908cd9b8e8,0.035891
3,dceb8ce861,0.021701
4,f4f0ac6b06,0.144503
...,...,...
60007,4762fb04c5,0.011551
60008,533d1516e7,0.052036
60009,c93b21a707,-0.012099
60010,3fe0668de5,0.051307


In [326]:
sm_score

0.008247578660209256